In [1]:
from yargy.labels import (
    gram,
    gram_not,
    gram_not_in,
    gram_any,
    dictionary,
    dictionary_not,
    not_eq,
    or_,
    label,
    is_capitalized,
    gnc_match,
    type_required,
    string_type
)
from yargy.compat import string_type
from yargy.parser import OR
from natasha.grammars.organisation.interpretation import OrganisationObject

In [2]:
from natasha.grammars.pipelines import CommercialOrganisationPipeline
from natasha.grammars.pipelines import AbbreviationalOrganisationPipeline

new_dictionary_com = {
        "авиакомпания",
        "госкомпания",
        "инвесткомпания",
        "медиакомпания",
        "оффшор-компания",
        "радиокомпания",
        "телекомпания",
        "телерадиокомпания",
        "траст-компания",
        "фактор-компания",
        "холдинг-компания",
        "энергокомпания",
        "компания-производитель",
        "компания-изготовитель",
        "компания-заказчик",
        "компания-исполнитель",
        "компания-посредник",
        "группа_управляющий_компания",
        "агрофирма",
        "турфирма",
        "юрфирма",
        "фирма-производитель",
        "фирма-изготовитель",
        "фирма-заказчик",
        "фирма-исполнитель",
        "фирма-посредник",
        "авиапредприятие",
        "агропредприятие",
        "госпредприятие",
        "нацпредприятие",
        "промпредприятие",
        "энергопредприятие",
        "авиакорпорация",
        "госкорпорация",
        "профорганизация",
        "стартап",
        "нотариальный_контора",
        "букмекерский_контора",
        "авиазавод",
        "автозавод",
        "винзавод",
        "подстанция",
        "гидроэлектростанция"
}

new_dictionary_abbr = {
    'ик',
    'нк',
    'хк',
    'ип',
    'чп',
    'ичп',
    'гпф',
    'нпф',
    'бф',
    'спао',
    'сро',
}

new_dictionary_com.update(CommercialOrganisationPipeline.Dictionary)
new_dictionary_abbr.update(AbbreviationalOrganisationPipeline.Dictionary)

from yargy.pipeline import CustomGrammemesPipeline

class NewCommercialOrganisationPipeline(CustomGrammemesPipeline):
    Grammemes = {
        'Orgn/Commercial',
    }
    Dictionary = new_dictionary_com
    
class NewAbbreviationalOrganisationPipeline(CustomGrammemesPipeline):
    Grammemes = {
        'Orgn/Abbr',
    }
    Dictionary = new_dictionary_abbr

In [3]:
#Россельхозбанк, Абсолют Банк, СЕВЕРГАЗБАНК

from yargy import Grammar, Parser

@label
@type_required(string_type)
def is_bank(bank, token, value):
    return ('банк' in token.value or 'Банк' in token.value or 'БАНК' in token.value) == bank

bank_grammar = Grammar('Bank', [
    {
        'labels': [
            is_bank(False),
            is_capitalized(True),
        ],
        'optional': True,
        'repeatable': True,
    },
    {
        'labels': [
            is_bank(True),
            is_capitalized(True),
        ],
    },
    {
        'labels': [
            is_bank(False),
            is_capitalized(True),
        ],
        'optional': True,
        'repeatable': True,
    },
])

In [4]:
#президент НАПФ

POSITION = {
    "президент",
    "сопрезидент",
    "вице-президент",
    "экс-президент",
    "председатель",
    "руководитель",
    "директор",
    "глава",
}

@label
@type_required(string_type)
def is_abbr(case, token, value):
    return token.value.isupper() == case

prob_org_grammar = Grammar('NonQuoted', [
        {
            'labels': [
                dictionary(POSITION),
            ],
        },
        {
            'labels': [
                is_abbr(True),
            ],
            'interpretation': {
                'attribute': OrganisationObject.Attributes.Name,
            },
        },
    ]
)

In [5]:
#комментируют в Intouch

VERB_COMMERCIAL = {
    "сообщить",
    "комментировать",
    "прокомментировать",
    "объяснить",
    "пояснить",
    "разъяснить",
    "заявить",
    "информировать",
    "проинформировать",
}

latin_grammar = Grammar('Latin', [
        {
            'labels': [
                dictionary(VERB_COMMERCIAL),
            ],
        },
        {
            'labels': [
                dictionary("в"),
            ],
            'optional': True, 
        },
        {
            'labels': [
                gram_any({
                        'LATN',
                        'NUMBER',
                    }),
            ],
            'repeatable': True,
            'interpretation': {
                'attribute': OrganisationObject.Attributes.Name,
            },
        },
    ]
)

In [6]:
#Google Inc, EG Capital Partners

ORGN_COMMERCIAL_LATIN = {
    "ag",
    "co",
    "corp",
    "eg",
    "gmbh",
    "ibc",
    "inc",
    "kg",
    "llc",
    "lp",
    "llp",
    "ltd",
    "plc",
    "s.a.",
    "corporation",
    "incorporated",
    "limited",
}

latin_abbr_grammar = Grammar('OfficialLatin', [
    {
        'labels': [
            dictionary_not(ORGN_COMMERCIAL_LATIN),
            is_capitalized(True),
            gram_any({
                    'LATN',
                    'NUMBER',
            }),
            
        ],
        'optional': True,
        'repeatable': True,
    },
    {
        'labels': [
            dictionary(ORGN_COMMERCIAL_LATIN),
        ],
    },
    {
        'labels': [
            dictionary_not(ORGN_COMMERCIAL_LATIN),
            is_capitalized(True),
            gram_any({
                    'LATN',
                    'NUMBER',
            }),
            
        ],
        'optional': True,
        'repeatable': True,
    },
])

In [7]:
#Интрейд Корпорейшн

ORGN_COMMERCIAL_LATIN_KIR = {
    "аг",
    "ко",
    "корп",
    "гмбх",
    "инк",
    "ллс",
    "лтд",
    "корпорэйшн",
    "корпорейшн",
    "инкорпорэйтед",
    "инкорпорейтед",
    "лимитэд",
    "лимитед",
}

latin_abbr_grammar = Grammar('OfficialNonLatin', [
    {
        'labels': [
            dictionary_not(ORGN_COMMERCIAL_LATIN_KIR),
            is_capitalized(True),
        ],
        'repeatable': True,
    },
    {
        'labels': [
            dictionary(ORGN_COMMERCIAL_LATIN_KIR),
        ],
    },
])